In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")
# Enable interactive plotting in Jupyter Notebook

from utils.camera_utils import *

In [2]:
run_name = "RUN3"

spiegel_calib = ["spiegel.mp4", "spiegel1.mp4", "spiegel2.mp4"]
front_calib = ["front.mp4", "front1.mp4", "front2.mp4"]
stereo_calib = ["stereo.mp4", "stereo2.mp4", "stereo3.mp4", "stereo4.mp4", "stereo5.mp4", "stereo6.mp4", "stereo7.mp4"]

In [3]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
sC = stereoCamera(name=run_name, camera_size={0: (480, 240), 1: (480, 240)})
vL = videoLoader()


# Camera Positions

In [ ]:
vL.load_video(os.path.join(detection_dir, detection_files[-1]))
frame = vL[10]
# 0 = mirror-cam 1 = frontal-cam
sC.set_anchor_point(frame, 0)
sC.set_anchor_point(frame, 1)
for frame in vL[:100]:
    frame = sC.draw_camera_region(frame)
    cv2.imshow("Frame", frame)
    frame0, frame1 = sC(frame)
    cv2.waitKey(10)
cv2.destroyAllWindows()


# Mirror Calibration

In [ ]:
frames = []
for file in spiegel_calib:
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100:110])
    
sC.calibrate(frames,0, rows=8, columns=10)

# Frontal Calibration

In [ ]:
frames = []
for file in front_calib:
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100:110])
      
sC.calibrate(frames,1, rows=8, columns=10)

# Stereo Calibration

In [4]:
frames = []

for file in stereo_calib:
    print(os.path.join(calibration_dir, file))
    vL.load_video(os.path.join(calibration_dir, file))
    frames.append(vL[100])
sC.load_from_yaml(f"{run_name}.yaml")
objectpoints, imgpoints_1, imgpoints_2, images = sC.stereo_calibrate(frames[::4], rows=8, columns=10, undistort=True)

../videos/RUN3/calibration/stereo.mp4
../videos/RUN3/calibration/stereo2.mp4
../videos/RUN3/calibration/stereo3.mp4
../videos/RUN3/calibration/stereo4.mp4
../videos/RUN3/calibration/stereo5.mp4
../videos/RUN3/calibration/stereo6.mp4
../videos/RUN3/calibration/stereo7.mp4


QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread

Current line: [651, 221]
Current line: [651, 221, 665, 302]
Current line: [706, 255]
Current line: [706, 255, 614, 290]
Removing last line
Removing last line
Current line: [637, 241]
Current line: [637, 241, 651, 312]
Current line: [714, 301]
Current line: [714, 301, 719, 248]
Removing last line
Removing last line
Current line: [612, 208]
Current line: [612, 208, 592, 387]
Current line: [634, 388]
Current line: [634, 388, 650, 207]
Current line: [689, 208]
Current line: [689, 208, 672, 389]
Current line: [715, 389]
Current line: [715, 389, 727, 210]
Current line: [765, 211]
Current line: [765, 211, 753, 389]
Current line: [796, 390]
Current line: [796, 390, 801, 207]
Current line: [840, 210]
Current line: [840, 210, 838, 394]
Current line: [880, 390]
Current line: [880, 390, 879, 210]
Current line: [919, 207]
Current line: [919, 207, 918, 389]
Current line: [960, 368]
Current line: [960, 368, 557, 366]
Current line: [558, 339]
Current line: [558, 339, 957, 344]
Current line: [953, 317]

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread

Current line: [632, 148]
Current line: [632, 148, 614, 506]
Current line: [689, 507]
Current line: [689, 507, 696, 145]
Current line: [760, 148]
Current line: [760, 148, 761, 507]
Current line: [836, 508]
Current line: [836, 508, 823, 152]
Current line: [886, 151]
Current line: [886, 151, 909, 506]
Current line: [983, 505]
Current line: [983, 505, 949, 146]
Current line: [1014, 149]
Current line: [1014, 149, 1058, 507]
Current line: [1132, 509]
Current line: [1132, 509, 1078, 148]
Current line: [1139, 147]
Current line: [1139, 147, 1206, 510]
Current line: [1271, 450]
Current line: [1271, 450, 546, 456]
Current line: [553, 406]
Current line: [553, 406, 1261, 399]
Current line: [1247, 352]
Current line: [1247, 352, 558, 356]
Current line: [560, 310]
Current line: [560, 310, 1237, 304]
Current line: [1229, 266]
Current line: [1229, 266, 565, 269]
Current line: [569, 226]
Current line: [569, 226, 1217, 222]
Current line: [1210, 183]
Current line: [1210, 183, 566, 185]


...

Current line: [692, 132]
Current line: [692, 132, 657, 343]
Current line: [700, 350]
Current line: [700, 350, 726, 137]
Current line: [766, 143]
Current line: [766, 143, 738, 358]
Current line: [780, 362]
Current line: [780, 362, 803, 152]
Current line: [842, 156]
Current line: [842, 156, 819, 370]
Current line: [860, 377]
Current line: [860, 377, 881, 164]
Current line: [918, 171]
Current line: [918, 171, 900, 382]
Current line: [940, 391]
Current line: [940, 391, 955, 178]
Current line: [994, 182]
Current line: [994, 182, 980, 393]
Current line: [1019, 373]
Current line: [1019, 373, 620, 306]
Current line: [627, 281]
Current line: [627, 281, 1022, 346]
Current line: [1023, 317]
Current line: [1023, 317, 631, 252]
Current line: [640, 227]
Current line: [640, 227, 1024, 291]
Current line: [1024, 263]
Current line: [1024, 263, 642, 200]
Current line: [647, 175]
Current line: [647, 175, 1028, 237]
Current line: [1028, 212]
Current line: [1028, 212, 651, 150]


QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread

Current line: [536, 189]
Current line: [536, 189, 469, 493]
Current line: [540, 487]
Current line: [540, 487, 600, 180]
Current line: [662, 177]
Current line: [662, 177, 609, 485]
Current line: [685, 477]
Current line: [685, 477, 723, 168]
Current line: [787, 163]
Current line: [787, 163, 760, 472]
Current line: [833, 464]
Current line: [833, 464, 849, 156]
Current line: [912, 152]
Current line: [912, 152, 911, 455]
Current line: [984, 451]
Current line: [984, 451, 979, 145]
Current line: [1041, 133]
Current line: [1041, 133, 1061, 448]
Current line: [1128, 392]
Current line: [1128, 392, 410, 455]
Current line: [421, 412]
Current line: [421, 412, 1126, 348]
Current line: [1124, 307]
Current line: [1124, 307, 431, 371]
Current line: [445, 331]
Current line: [445, 331, 1121, 265]
Current line: [1114, 228]
Current line: [1114, 228, 454, 297]
Current line: [464, 256]
Current line: [464, 256, 1109, 191]
Current line: [1108, 157]
Current line: [1108, 157, 469, 222]


QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread (0x85e9a50).
Cannot move to target thread (0x824ea10)

QObject::moveToThread: Current thread (0x824ea10) is not the object's thread

TypeError: show_and_switch() missing 2 required positional arguments: 'rows' and 'columns'

In [ ]:
vL.load_video(os.path.join(calibration_dir, stereo_calib[0]))
cv2.imshow("frame", vL[0])
cv2.waitKey(0)

In [ ]:
sC.conf["camera_matrix"][1]

In [ ]:
sC.conf

# Save Calibration to YAML

In [ ]:
sC.save_to_yaml(f"{run_name}.yaml")

sC.save_to_yaml(f"{run_name}_exp.yaml")

In [ ]:
sC.load_from_yaml(f"{run_name}.yaml")

In [ ]:
sC.conf["anchor_point"], sC.name